In [4]:
import numpy as np

def somme_des_logs(y):
    sommes = y.copy()
    for i in range(len(y)):
        for j in range(len(y[0])):
            sommes[i][j] = som_des_logs(y[i][j])
    return sommes
 



In [5]:
import math

def som_des_logs(n):
    somme = 0
    for i in range(1, n+1):
        somme += math.log(i)
    return somme


In [122]:
import numpy as np
import scipy.special

def salm(nchain, x, y, alpha, beta, gamma, tau, lambd, prop_sd):
  
    # Initialisation
    # res = np.ones((nchain + 1, 4))
    res = []
    res.append([alpha, beta, gamma, 1/np.sqrt(tau)])
    res_lambda = []
    res_lambda.append(lambd)
    #print(res[0][1])
    n = len(x)
    m = y.shape[1]
    logmu = np.zeros((n,m))
    for l in range(n):
        for k in range(m):
            logmu[l, k] = alpha + beta * np.log(10 + x[l]) + gamma * x[l] + lambd[l, k]
    mu = np.exp(logmu)

    
    acc_rates = np.zeros(3) 
    
    print(res)
    print(res_lambda)
    for i in range(nchain):

        lambd = res_lambda[i]
        # Mise à jour de alpha
        alpha = res[i][0]
        prop = np.random.normal(alpha, prop_sd[0])  # marche aléatoire simple
        prop_logmu = np.zeros((n,m))
        for l in range(n):
            for k in range(m):
                prop_logmu[l, k] = prop + beta * np.log(10 + x[l]) + gamma * x[l] + lambd[l, k]
        if i<= 10:
            print(alpha, beta, gamma, lambd[1,1])
            print(prop_logmu[1,1])
        prop_mu = np.exp(prop_logmu)
        sum_log = somme_des_logs(y)
        top = - ((prop**2) / (2 * 1e-6)) + np.sum(y * np.log(prop_mu+1e-10) - sum_log - prop_mu)
        bottom = - ((alpha**2) / (2 * 1e-6)) + np.sum(y * np.log(mu) - sum_log - mu)
        acc_prob = np.exp(top - bottom)  # la proportion des noyaux est de 1 (symétrie du noyau)
        
        if np.random.uniform(0, 1) < min(1, acc_prob):
            alpha = prop
            logmu = prop_logmu
            mu = prop_mu
            acc_rates[0] += 1
        
        # Mise à jour de beta
        print(i)
        beta = res[i][1]
        prop = np.random.normal(beta, prop_sd[1])  # marche aléatoire simple
        for l in range(n):
            for k in range(m):
                prop_logmu[l, k] = alpha + prop * np.log(10 + x[l]) + gamma * x[l] + lambd[l, k]
        prop_mu = np.exp(prop_logmu)
        sum_log = somme_des_logs(y)
        top = - ((prop**2) / (2 * 1e-6)) + np.sum(y * np.log(prop_mu+1e-10) - sum_log - prop_mu)
        bottom = - ((beta**2) / (2 * 1e-6)) + np.sum(y * np.log(mu) - sum_log - mu)
        acc_prob = np.exp(top - bottom)  # la proportion des noyaux est de 1 (symétrie du noyau)
        
        if np.random.uniform(0, 1) < min(1, acc_prob):
            beta = prop
            logmu = prop_logmu
            mu = prop_mu
            acc_rates[1] += 1
        
        # Mise à jour de gamma
        gamma = res[i][2]
        prop = np.random.normal(gamma, prop_sd[2])  # marche aléatoire simple
        for l in range(n):
            for k in range(m):
                prop_logmu[l, k] = alpha + beta * np.log(10 + x[l]) + prop * x[l] + lambd[l, k]
        prop_mu = np.exp(prop_logmu)
        sum_log = somme_des_logs(y)
        top = - ((prop**2) / (2 * 1e-6)) + np.sum(y * np.log(prop_mu+1e-10) - sum_log - prop_mu)
        bottom = - ((gamma**2) / (2 * 1e-6)) + np.sum(y * np.log(mu) - sum_log - mu)
        acc_prob = np.exp(top - bottom)  # la proportion des noyaux est de 1 (symétrie du noyau)
        
        if np.random.uniform(0, 1) < min(1, acc_prob):
            gamma = prop
            logmu = prop_logmu
            mu = prop_mu
            acc_rates[2] += 1
        
        # Mise à jour de tau
        tau = np.random.gamma(1e-3 + (n + m) / 2, 1e-3 + 0.5 * np.sum(lambd**2))

        # Mise à jour de lambd
        for k in range(n):
            for l in range(m):
                prop = np.random.normal(lambd[k,l], prop_sd[3])
                for o in range(n):
                    for p in range(m):
                        prop_logmu[o, p] = alpha + beta * np.log(10 + x[o]) + prop * x[p] + lambd[o,p]
                prop_mu = np.exp(prop_logmu)
                top = - (prop**2 / (2*tau))- prop_mu[k,l] + np.log(prop_mu[k,l]**y[k, l]) - som_des_logs(y[k][l])
                bottom = - (lambd[k, l]**2/ (2*tau))- mu[k,l] + np.log(mu[k, l]**y[k, l]) - som_des_logs(y[k][l])
                acc_prob = np.exp(top - bottom)
              
                if np.random.uniform(0, 1) < min(1, acc_prob):
                    lambd[k, l] = prop
                    logmu= prop_logmu
                    mu = prop_mu
        
        # Mise à jour de la chaîne de Markov et de b
        res.append([alpha, beta, gamma, 1/np.sqrt(tau)])
        res_lambda.append(lambd)
    
    my_list = {"chain": res, "acc_rates": acc_rates}
    return my_list



In [118]:
lambd

array([[-3.12145899, -2.77887879, -0.5601739 ],
       [ 7.45065847, -2.18254668,  1.9544398 ],
       [-6.73812765, -5.77324897, -4.48847661],
       [ 0.14810958, -2.7188603 ,  1.00196847],
       [-4.3061934 ,  1.02458031, -0.67306985],
       [ 2.50175555,  0.51861174,  4.71922444]])

In [133]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, gamma, poisson

# Récupération des données
Ndoses = 6
Nplates = 3
y = np.array([[15, 16, 16],
              [27, 33, 20],
              [21, 18, 26],
              [41, 38, 27],
              [29, 21, 33],
              [60, 41, 42]])
x = np.array([0, 10, 33, 100, 333, 1000])

# Initialisation
alpha = 0.0
beta = 0
gamma = 0
tau = 0.1
lambd = np.random.normal(loc=0, scale=np.sqrt(1/tau), size=(6, 3))

resultat = salm(int(1e4), x, y, alpha, beta, gamma, tau, lambd, prop_sd=[0.03, 0.03, 0.03, 0.03])  # prop_sd choisi pour avoir une apparence de chaîne cohérente

resultat_chain = resultat["chain"][1000:]  # enlever les 1000 premiers (burn-in)

moychain = np.mean(resultat_chain, axis=0)  # moyenne
sdchain = np.std(resultat_chain, axis=0)  # écart-type

print("alpha est estimé à", moychain[0], "avec un écart-type de", sdchain[0])
print("beta est estimé à", moychain[1], "avec un écart-type de", sdchain[1])
print("gamma est estimé à", moychain[2], "avec un écart-type de", sdchain[2])
print("sigma est estimé à", moychain[3], "avec un écart-type de", sdchain[3])



[[0.0, 0, 0, 3.162277660168379]]
[array([[ 5.59472061, -3.65998423,  2.99406133],
       [-1.26654916, -2.93660081,  2.67810902],
       [ 2.39706003,  0.16205732,  1.8368802 ],
       [ 3.20664815, -1.17007492, -5.16749894],
       [-0.74402506, -1.60359356,  0.49868725],
       [ 4.88448747, -0.39202089, -3.57197854]])]
0.0 0 0 -2.9366008072269487
-2.922837890692745
0
0.0 0 0 -2.9366008072269487
-2.97800734253725
1
-0.04140653531030115 0 0 -2.9366008072269487
-2.9856924779511425
2
-0.04909167072419383 0 0 -2.9366008072269487
-2.97791240145685
3
-0.041311594229900966 0 0 -2.9366008072269487
-2.9401230683191364
4
-0.0035222610921874647 0 0 -2.9366008072269487
-2.9827483591895048
5
-0.04614755196255615 0 0 -2.9366008072269487
-2.964729823451706
6
-0.028129016224757487 0 0 -2.9366008072269487
-2.9495596227925813
7
-0.012958815565632702 0.0006686394828384437 0 -2.9366008072269487
-2.930969467339592
8
0.003628275009244942 0.0006686394828384437 0 -2.9366008072269487
-2.973053742779997
9
-0.

C:\Users\tetoflex\AppData\Local\Temp\ipykernel_19288\223682222.py:100: RuntimeWarning: overflow encountered in scalar power
  bottom = - (lambd[k, l]**2/ (2*tau))- mu[k,l] + np.log(mu[k, l]**y[k, l]) - som_des_logs(y[k][l])
C:\Users\tetoflex\AppData\Local\Temp\ipykernel_19288\223682222.py:99: RuntimeWarning: overflow encountered in scalar power
  top = - (prop**2 / (2*tau))- prop_mu[k,l] + np.log(prop_mu[k,l]**y[k, l]) - som_des_logs(y[k][l])
C:\Users\tetoflex\AppData\Local\Temp\ipykernel_19288\223682222.py:101: RuntimeWarning: invalid value encountered in scalar subtract
  acc_prob = np.exp(top - bottom)
C:\Users\tetoflex\AppData\Local\Temp\ipykernel_19288\223682222.py:99: RuntimeWarning: divide by zero encountered in log
  top = - (prop**2 / (2*tau))- prop_mu[k,l] + np.log(prop_mu[k,l]**y[k, l]) - som_des_logs(y[k][l])
C:\Users\tetoflex\AppData\Local\Temp\ipykernel_19288\223682222.py:100: RuntimeWarning: divide by zero encountered in log
  bottom = - (lambd[k, l]**2/ (2*tau))- mu[k,l

95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
3

C:\Users\tetoflex\AppData\Local\Temp\ipykernel_19288\223682222.py:62: RuntimeWarning: overflow encountered in exp
  acc_prob = np.exp(top - bottom)  # la proportion des noyaux est de 1 (symétrie du noyau)


1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794
1795
1796
1797
1798
1799
1800
1801
1802
1803
1804
1805
1806
1807
1808
1809
1810
1811
1812
1813
1814
1815
1816
1817
1818
1819
1820
1821
1822
1823
1824
1825
1826
1827
1828
1829
1830
1831
1832
1833
1834
1835
1836
1837
1838
1839
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893


C:\Users\tetoflex\AppData\Local\Temp\ipykernel_19288\223682222.py:80: RuntimeWarning: overflow encountered in exp
  acc_prob = np.exp(top - bottom)  # la proportion des noyaux est de 1 (symétrie du noyau)


1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093


In [11]:
print(resultat)

{'chain': [[0.0, 0, 0, 3.162277660168379], [0.0, 0, 0, 0.12461244843136624], [0.048511516993128445, 0, 0, 0.19815239504654927], [-0.2934574418780655, 0, 0, 0.11723156430033638], [0.09992619707771888, 0, 0, 0.0888041654240146], [0.21404109416912626, 0, 0, 0.13556265059801384], [0.21203598664740025, 0, 0, 0.14532482849938655], [0.3521692079082178, 0, 0, 0.12158236215593805], [0.2696035757117078, 0, 0, 0.10636251772598661], [0.7400860862470201, 0, 0, 0.12641085434567245], [0.7809317972201517, 0, 0.0032240154029589056, 0.10583776472970767], [0.7721239975114671, 0, 0.0032240154029589056, 0.11784272297828188], [0.998998709352358, 0, 0.0032240154029589056, 0.19655921772846474], [0.9195993089677875, 0, 0.0032240154029589056, 0.12161367992851814], [0.8185480359718877, 0, 0.0032240154029589056, 0.20555345673691647], [0.6879838087969318, 0, 0.0032240154029589056, 0.18314889293511719], [0.730745907632573, 0, 0.0032240154029589056, 0.10769704013638828], [0.7107581134501385, 0, 0.0032240154029589056

In [164]:

# Implémentation de la méthode MCMC (Hastings within Gibbs)
def model(y, x, nchain, alpha, beta, gamma, tau, lambda_):
    n = len(x)
    m = y.shape[1]
    res = np.zeros((nchain, 4))
    for k in range(nchain):
        for i in range(n):
            for j in range(m):
                mu_ij = np.exp(alpha + beta * np.log(x[i] + 10) + gamma * x[i] + lambda_[i, j])
                lambda_[i, j] = np.random.normal(0, np.sqrt(1/tau))
                y[i, j] = np.random.poisson(mu_ij)
        alpha = np.random.normal(alpha_star, np.sqrt(1/1e6))
        beta = np.random.normal(0, np.sqrt(1/1e6))
        gamma = np.random.normal(0, np.sqrt(1/1e6))
        tau = np.random.gamma(0.001 + 9 * n * m, 0.001 + 0.5 * np.sum(lambda_**2))
        res[k] = [alpha, beta, gamma, 1/np.sqrt(tau)]
    return res

results = model(y, x, 10000, alpha_star, beta, gamma, tau, lambd)

# Récupération des résultats
burn_in = 1000
results_post_burn_in = results[burn_in:, :]
moychain = np.mean(results_post_burn_in, axis=0) # moyenne
sdchain = np.std(results_post_burn_in, axis=0) # écart type

print("Alpha est estimé à", moychain[0], "avec un écart-type de", sdchain[0])
print("Beta est estimé à", moychain[1], "avec un écart-type de", sdchain[1])
print("Gamma est estimé à", moychain[2], "avec un écart-type de", sdchain[2])
print("Sigma est estimé à", moychain[3], "avec un écart-type de", sdchain[3])

# Visualisation des résultats
plt.figure(figsize=(10, 10))

# Allure des densités pour Alpha, Beta, Gamma et Sigma
plt.subplot(2, 2, 1)
plt.title("Density de Alpha")
plt.hist(results_post_burn_in[:, 0], bins=30, density=True)
plt.subplot(2, 2, 2)
plt.title("Density de Beta")
plt.hist(results_post_burn_in[:, 1], bins=30, density=True)
plt.subplot(2, 2, 3)
plt.title("Density de Gamma")
plt.hist(results_post_burn_in[:, 2], bins=30, density=True)
plt.subplot(2, 2, 4)
plt.title("Density de Sigma")
plt.hist(results_post_burn_in[:, 3], bins=30, density=True)

plt.tight_layout()
plt.show()


NameError: name 'lambda_' is not defined

In [2]:
mean = [moychain[0], moychain[1], moychain[2], moychain[3]]
sd = [sdchain[0], sdchain[1], sdchain[2], sdchain[3]]

In [218]:
#Baseline here indicates Breslow and Clayton estimates
def plot_parameter_comparison(params_df):
    ind = np.arange(len(params_df))  # the x locations for the groups
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(ind - width / 2, params_df['mean'], width, label='Model', yerr=params_df['std'])
    rects2 = ax.bar(ind + width / 2, params_df['mean_base'], width, label='Baseline', yerr=params_df['std_base'])

    # Add some text for labels, title, and custom x-axis tick labels
    ax.set_ylabel('Values')
    ax.set_title('Model vs. baseline Parameter Comparison')
    ax.set_xticks(ind)
    ax.set_xticklabels(['alpha', 'beta', 'gamma', 'sigma'])
    ax.legend()

    plt.show()

#results_summary = results.describe().T
import pandas as pd
params_df = pd.DataFrame()
params_df['mean'] = mean
params_df['std'] = sd
#Breslow and Clayton estimates
params_df['mean_base'] = [2.193, 0.3059, 69.577*(10**(-4)), 0.2608]
params_df['std_base'] = [0.3874, 0.1054, 4.525*(10**(-4)), 0.08077]
params_df.reset_index(drop=True, inplace=True)

plot_parameter_comparison(params_df)

NameError: name 'mean' is not defined